In [5]:
import pandas as pd
import numpy as np

from sklearn.metrics import cohen_kappa_score

In [4]:
df = pd.read_csv("processed_annotations/CANNONICAL_LIST.csv")
df.head()

,Unnamed: 0,ICD_doc_id,sentence,num_words,num_chars,sentence_string,KATHLEEN_A,KATHLEEN_B,KATHLEEN_text,annotation_id,...,LIZ_B,LIZ_text,KAYCEE_A,KAYCEE_B,KAYCEE_text,inclusion,Sum_Annotations,ANY_A,ANY_B,EITHER_AB
0,0,95581557,Ct) MERCY HEALTH \n,5,18,Ct) MERCY HEALTH,0,0,NaN,NaN,...,0,NaN,0,0,NaN,0,0,0,0,0
1,1,95581557,"GRAND RAPIDS,",3,13,"GRAND RAPIDS,",0,0,NaN,NaN,...,0,NaN,0,0,NaN,0,0,0,0,0
2,2,95581557,Ml \nCONSENT,3,11,Ml CONSENT,0,0,NaN,NaN,...,0,NaN,0,0,NaN,0,0,0,0,0
3,3,95581557,TO RECEIVE \n,3,12,TO RECEIVE,0,0,NaN,NaN,...,0,NaN,0,0,NaN,0,0,0,0,0
4,4,95581557,Patient Label \n\nNON-EMERGENCY TRANSFUSION OF...,9,50,Patient Label NON-EMERGENCY TRANSFUSION OF BLOOD,0,0,NaN,NaN,...,0,NaN,0,0,NaN,1,0,0,0,0


In [27]:
included = df[(df['inclusion'] == 1)]
annotated_AB = df[(df['EITHER_AB'] == 1)]
annotated_A = df[(df['ANY_A'] == 1)]
annotated_B = df[(df['ANY_B'] == 1)]

print(f"Total Sentences Detected: {len(df)}")
print(f"Total Included Sentences Detected: {len(included)}; (% total: {len(included)/len(df):.3f})")
print(f"Annotated Either A or B: {len(annotated_AB)}; (% total: {len(annotated_AB)/len(df):.3f}); (% included: {len(annotated_AB)/len(included):.3f})")
print(f"Annotated A: {len(annotated_A)}; (% total: {len(annotated_A)/len(df):.3f}); (% included: {len(annotated_A)/len(included):.3f})")
print(f"Annotated B: {len(annotated_B)};(% total: {len(annotated_B)/len(df):.3f}); (% included: {len(annotated_B)/len(included):.3f})")

print()
N_docs = len(set(df['ICD_doc_id']))

KATHLEEN_A = np.sum(df['KATHLEEN_A'])
KATHLEEN_B = np.sum(df['KATHLEEN_B'])

LIZ_A = np.sum(df['LIZ_A'])
LIZ_B = np.sum(df['LIZ_B'])

KAYCEE_A = np.sum(df['KAYCEE_A'])
KAYCEE_B = np.sum(df['KAYCEE_B'])

print(f"KATHLEEN A: {KATHLEEN_A} ({KATHLEEN_A/N_docs:.3f} per doc), B:{KATHLEEN_B} ({KATHLEEN_B/N_docs:.3f} per doc)")
print(f"LIZ A: {LIZ_A}  ({LIZ_A/N_docs:.3f} per doc), B:{LIZ_B}  ({LIZ_B/N_docs:.3f} per doc)")
print(f"KAYCEE A: {KAYCEE_A}  ({KAYCEE_A/N_docs:.3f} per doc), B:{KAYCEE_B}  ({KAYCEE_B/N_docs:.3f} per doc)")

Total Sentences Detected: 10941
Total Included Sentences Detected: 6415; (% total: 0.586)
Annotated Either A or B: 688; (% total: 0.063); (% included: 0.107)
Annotated A: 595; (% total: 0.054); (% included: 0.093)
Annotated B: 94;(% total: 0.009); (% included: 0.015)

KATHLEEN A: 135 (1.007 per doc), B:36 (0.269 per doc)
LIZ A: 119  (0.888 per doc), B:31  (0.231 per doc)
KAYCEE A: 341  (2.545 per doc), B:27  (0.201 per doc)


In [29]:
KATHLEENvLIZ_A = cohen_kappa_score(included['KATHLEEN_A'], included['LIZ_A'])
KATHLEENvKAYCEE_A = cohen_kappa_score(included['KATHLEEN_A'], included['KAYCEE_A'])
LIZvKAYCEE_A = cohen_kappa_score(included['LIZ_A'], included['KAYCEE_A'])

KATHLEENvLIZ_B = cohen_kappa_score(included['KATHLEEN_B'], included['LIZ_B'])
KATHLEENvKAYCEE_B = cohen_kappa_score(included['KATHLEEN_B'], included['KAYCEE_B'])
LIZvKAYCEE_B = cohen_kappa_score(included['KAYCEE_B'], included['LIZ_B'])

print(f"KATHLEENvLIZ_A: {KATHLEENvLIZ_A:.4f}")
print(f"KATHLEENvKAYCEE_A: {KATHLEENvKAYCEE_A:.4f}")
print(f"LIZvKAYCEE_A: {LIZvKAYCEE_A:.4f}")
print(f"KATHLEENvLIZ_B: {KATHLEENvLIZ_B:.4f}")
print(f"KATHLEENvKAYCEE_B: {KATHLEENvKAYCEE_B:.4f}")
print(f"LIZvKAYCEE_B: {LIZvKAYCEE_B:.4f}")

KATHLEENvLIZ_A: -0.0201
KATHLEENvKAYCEE_A: -0.0311
LIZvKAYCEE_A: -0.0283
KATHLEENvLIZ_B: -0.0052
KATHLEENvKAYCEE_B: -0.0048
LIZvKAYCEE_B: -0.0045
